In [323]:
#定数エリア
DIFF_RATIO = 1 #荷物判定の割合(% 3%なら3を指定)



#インポートエリア
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
from pathlib import Path, PurePosixPath

#jupyter用
%matplotlib inline


In [325]:
def set_box(file:Path) -> dict:

    img = cv2.imread(str(file))

    box = dict()
    # 各ボックスの座標データ　左上から縦に１列ずつ合計40ボックス
    # 読み方：img[Y座標上:Y座標下, X座標左, X座標右]
    box[0]=img[1039:1296,96:885]
    box[1]=img[1309:1558,110:906]
    box[2]=img[1570:1810,142:916]
    box[3]=img[1816:2055,168:936]
    box[4]=img[2066:2294,193:948]
    box[5]=img[2303:2526,213:970]
    box[6]=img[2533:2747,245:982]
    box[7]=img[2761:2980,268:996]
    box[8]=img[1034:1290,920:1692]
    box[9]=img[1292:1553,936:1696]
    box[10]=img[1558:1795,950:1712]
    box[11]=img[1800:2041,970:1713]
    box[12]=img[2048:2281,984:1722]
    box[13]=img[2288:2512,990:1729]
    box[14]=img[2518:2734,1005:1732]
    box[15]=img[2748:2969,1018:1738]
    box[16]=img[1025:1284,1728:2486]
    box[17]=img[1291:1536,1729:2482]
    box[18]=img[1542:1782,1742:2476]
    box[19]=img[1784:2022,1746:2488]
    box[20]=img[2030:2260,1766:2472]
    box[21]=img[2264:2492,1756:2474]
    box[22]=img[2496:2724,1762:2468]
    box[23]=img[2724:2936,1768:2470]
    box[24]=img[1011:1268,2532:3285]
    box[25]=img[1272:1524,2524:3286]
    box[26]=img[1520:1776,2520:3248]
    box[27]=img[1778:2024,2520:3248]
    box[28]=img[2012:2252,2520:3230]
    box[29]=img[2244:2494,2516:3224]
    box[30]=img[2482:2708,2514:3210]
    box[31]=img[2712:2940,2508:3204]
    box[32]=img[994:1248,3316:4028]
    box[33]=img[1248:1508,3314:4020]
    box[34]=img[1510:1756,3300:4018]
    box[35]=img[1764:1998,3282:4012]
    box[36]=img[2004:2240,3274:3998]
    box[37]=img[2240:2468,3264:3964]
    box[38]=img[2482:2698,3246:3964]
    box[39]=img[2688:2910,3234:3954]
    return box

In [327]:
def make_emptyimage(box:dict):
    i = 0
    for chi_img in box.values():
        filename = Path(f'../images/empty/empty{i}.jpeg').resolve()
        cv2.imwrite(str(filename),chi_img)
        i += 1

In [328]:
def make_sourceimage(box:dict):
    i = 0
    for chi_img in box.values():
        filename = Path(f'../images/source/source{i}.jpeg').resolve()
        cv2.imwrite(str(filename),chi_img)
        i += 1

In [393]:
def export_result(result:list):
    import pandas as pd
    filename =  Path('./result/result.csv').resolve()
    if Path.exists(filename):
        res = pd.read_csv(filename,header=0)
        res = res.append(result)
        if len(res) > 4:
            res.drop(index=1)
        res.to_csv(filename)
    else:
        res = []
        res.append(result)
        res = pd.DataFrame(res)
        res.to_csv(filename)
    
    #3回連続のアラート情報がある場合、メッセージ表示
    if len(res) == 3:
        for i in range(len(res)):
            if res.iloc[0,i] and res.iloc[1,i] and res.iloc[2,i] :
                msg = f'{i} 番ポスト　郵送物あり'
                print(msg)

    
    


In [382]:
res

In [375]:
li = []

li.append([True,True,False])
li.append([True,True,False])
li.append([True,True,False])
li.append([True,True,False])
li.append([True,True,False])


while len(li) > 3:
    li.pop()


d = pd.DataFrame(li)
if len(d) == 3:
    for i in range(len(d)):
        if d.iloc[0,i] and d.iloc[1,i] and d.iloc[2,i] :
            msg = f'{i} 番ポスト　郵送物あり'
            print(msg)



0 番ポスト　郵送物あり
1 番ポスト　郵送物あり


In [329]:
def compare_img(img_1, img_2) -> float:
    """_summary_

    Args:
        img_1: 郵便受けの画像
        img_2: 空の郵便受けの画像

    Returns:
        float: 画像の差の割合
    Examples:
        diff_ratio = compare_img(cv2.imread('nimotsu_output3.jpg'),cv2.imread('empty_output3.jpg'))
    """
    import cv2
    import os
    import numpy as np


    height = img_1.shape[0]
    width = img_1.shape[1]

    img_size = (int(width), int(height))

    # # 画像をリサイズする
    image1 = cv2.resize(img_1, img_size)
    image2 = cv2.resize(img_2, img_size)

    # ２画像の差異を計算
    im_diff = image1.astype(int) - image2.astype(int)

    #変化総量　要素ごとの　BGRの値を足し合わせる
    ratio = (np.abs(im_diff).sum()) / (im_diff.size * 255)
    return(ratio, im_diff)

In [330]:
#初期化エリア
#空のボックス画像を指定
emptyfile = Path("../images/nimotsu0.jpeg").resolve()
# emptyimg = cv2.imread(str(emptyfile))
#空のボックス画像をボックス毎に切り分ける
empty_box = set_box(emptyfile)
#空のボックス画像を比較用として作成する
make_emptyimage(empty_box)

In [394]:

#実行エリア
#比較対象の画像を指定
file = Path("../images/nimotsuall.jpeg").resolve()

box = set_box(file)
make_sourceimage(box)


import pandas as pd
result = list()
for i in range(len(box)):
    ratio, img_diff =compare_img(box[i], empty_box[i])
    filename = Path(f'../images/difference/difference{i}.jpeg').resolve()
    cv2.imwrite(str(filename),img_diff)
    txt = f'i = {i}  画像相違割合 = {(ratio * 100):.3g} %'
    print(txt)
    result.append(ratio,ratio > DIFF_RATIO)
    
res = export_result(result)



i = 0  画像相違割合 = 3.22 %
i = 1  画像相違割合 = 1.85 %
i = 2  画像相違割合 = 1.34 %
i = 3  画像相違割合 = 2.03 %
i = 4  画像相違割合 = 4.49 %
i = 5  画像相違割合 = 6.36 %
i = 6  画像相違割合 = 7.03 %
i = 7  画像相違割合 = 6.6 %
i = 8  画像相違割合 = 10.6 %
i = 9  画像相違割合 = 10 %
i = 10  画像相違割合 = 7.03 %
i = 11  画像相違割合 = 6.01 %
i = 12  画像相違割合 = 5.87 %
i = 13  画像相違割合 = 6.37 %
i = 14  画像相違割合 = 6.46 %
i = 15  画像相違割合 = 7.69 %
i = 16  画像相違割合 = 11.3 %
i = 17  画像相違割合 = 10.1 %
i = 18  画像相違割合 = 9.95 %
i = 19  画像相違割合 = 11.2 %
i = 20  画像相違割合 = 11.2 %
i = 21  画像相違割合 = 11.1 %
i = 22  画像相違割合 = 11.6 %
i = 23  画像相違割合 = 11.6 %
i = 24  画像相違割合 = 7.92 %
i = 25  画像相違割合 = 7.42 %
i = 26  画像相違割合 = 8.8 %
i = 27  画像相違割合 = 12.8 %
i = 28  画像相違割合 = 14.9 %
i = 29  画像相違割合 = 14.9 %
i = 30  画像相違割合 = 16.7 %
i = 31  画像相違割合 = 14.9 %
i = 32  画像相違割合 = 4.79 %
i = 33  画像相違割合 = 7.46 %
i = 34  画像相違割合 = 5.8 %
i = 35  画像相違割合 = 9.81 %
i = 36  画像相違割合 = 10.2 %
i = 37  画像相違割合 = 11.6 %
i = 38  画像相違割合 = 12.3 %
i = 39  画像相違割合 = 12.9 %


C:\Users\user\AppData\Local\Temp\ipykernel_12812\4191727958.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(result)


In [251]:

    rects = find_rect_of_target_color(chi_img,5,overall_lower,overall_upper)
    area = 0
    if len(rects) > 0:
        
        # for rect in rects:
        rect = max(rects, key=(lambda x: x[2] * x[3]))
        
        if rect[2] > 0 and rect[3] > 0 : # 左上のｘ座標、左上のｙ座標、横幅、縦の幅）
        # if rect[1] > 1000 and rect[2] > 0 and rect[3] > 0 : # 左上のｘ座標、左上のｙ座標、横幅、縦の幅）

            cv2.rectangle(chi_img, tuple(rect[0:2]), tuple(rect[0:2] + rect[2:4]), (0, 0, 255), thickness=2)
            # cv2.putText(img,
            #             text=str(i) + ' x=' + str(rect[0]) + ' y=' + str(rect[1]),
            #             org=(rect[0], rect[1]),
            #             fontFace=cv2.FONT_HERSHEY_SIMPLEX,
            #             fontScale=4,
            #             color=(255, 0, 0),
            #             thickness=2,
            #             lineType=cv2.LINE_4)
    filename = f'output{i}.jpg'
    area =  rect[2] * rect[3]
    cv2.imwrite(filename,chi_img)
    if area < 100000:
        print('荷物発見',i, rect, rect[2] * rect[3])
    # print(rect)
    i += 1
